In [ ]:
!pip install pocketsphinx
!pip install https://huggingface.co/kormilitzin/en_core_med7_lg/resolve/main/en_core_med7_lg-any-py3-none-any.whl
!pip install gTTS
!pip install SpeechRecognition
!pip install python-docx
!pip install pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.4/607.4 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 18.8 MB/s eta 0:00:00
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.2
    Uninstalling wasabi-1.1.2:
      Successfully uninstalled wasabi-1.1.2
  Attempting uninstall: typer
    Found existing installation: typer 0.9.0
    Uninstalling typer-0.9.0:
      Successfully uninstalled typer-0.9.0
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python

In [ ]:
import speech_recognition as sr


def get_text_from_video(audio):
  # Initialize the recognizer
  recognizer = sr.Recognizer()

  # Load an audio file
  audio_file = "test-voice.wav"  # Replace with your audio file's path

  # Recognize speech using Google Web Speech API (online)
  with sr.AudioFile(audio_file) as source:
      audio = recognizer.record(source)
      try:
          recognized_text = recognizer.recognize_google(audio)
          return recognized_text
      except sr.UnknownValueError:
          print("Google Web Speech API could not understand audio.")
      except sr.RequestError as e:
          print("Could not request results from Google Web Speech API; {0}".format(e))


In [ ]:
text = get_text_from_video('/content/test-voice.wav')
print(text)

patient name is Ram age 19 tablet azithromycin 500 mg 3 days after food morning only syrup Robitussin 5 ml 5 days before food night done


In [ ]:
import spacy


def extract_details(text):
    med7 = spacy.load("en_core_med7_lg")

    # create distinct colours for labels
    col_dict = {}
    seven_colours = ['#e6194B', '#3cb44b', '#ffe119', '#ffd8b1', '#f58231', '#f032e6', '#42d4f4']
    for label, colour in zip(med7.pipe_labels['ner'], seven_colours):
        col_dict[label] = colour

    options = {'ents': med7.pipe_labels['ner'], 'colors':col_dict}

    doc = med7(text)

    #spacy.displacy.render(doc, style='ent', jupyter=True, options=options)

    return [(ent.text, ent.label_) for ent in doc.ents]

In [ ]:
print(extract_details(text))

In [ ]:
from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.shared import RGBColor, Pt
import datetime

def create_prescription(name, age, date, tablet):
    if len(name) == 0:
        print("Error: No patient name is found, please retry again!")
        raise ValueError('No name found')

    # Initialize the document
    document = Document()
    para = document.add_heading("")

    # Header
    runn = para.add_run("HealthCare Hospitals \n")
    run = para.add_run(" Dr.Rishi Raj M.D.(Neurology) \t \t Dr.Swathy M.S.(Ortho) \t Dr.Shreevarshann M.S.(Ophthalmology) \n No.20,Blossom Avenue,Madura \n Ph:0452-2588 1522 \n")

    # Set page attributes
    font = run.font
    runn.font.size = Pt(18)
    runn.font.color.rgb = RGBColor(153, 17, 150)
    font.size = Pt(16)
    font.color.rgb = RGBColor(217, 17, 213)
    para.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    para1 = document.add_paragraph()
    run1 = para1.add_run("Name: {}".format(name))
    run1.font.size = Pt(14)

    para2 = document.add_paragraph()
    run2 = para2.add_run("Age: {}".format(age))
    run2.font.size = Pt(14)

    para3 = document.add_paragraph()
    run3 = para3.add_run("Date: {}".format(date))
    run3.font.size = Pt(14)

    if len(tablet) == 0:
        print("Alert: Please include at least one tablet")
    else:
        table = document.add_table(len(tablet), 5)
        heading_cells = table.rows[0].cells
        heading_cells[0].text = 'Name'
        heading_cells[1].text = 'Dosage and Strength'
        heading_cells[2].text = 'Duration and Frequency'

        for i in range(len(tablet)):
            cells = table.add_row().cells
            cells[0].text = tablet[i][0]
            cells[1].text = tablet[i][1]
            cells[2].text = tablet[i][2]

    para5 = document.add_paragraph("\n \n")

    document.save("{}.docx".format(name))
    print("Successfully completed.")
    print("Success! Please verify the document generated and click upload.")
    print("Document Name: {}.docx".format(name))

# Example usage
name = "John Doe"
age = 35
date = datetime.date.today().strftime("%Y-%m-%d")
tablet = [("Medicine1", "5mg", "2 times a day"), ("Medicine2", "10mg", "1 time a day")]

create_prescription(name, age, date, tablet)


Successfully completed.
Success! Please verify the document generated and click upload.
Document Name: John Doe.docx


In [ ]:
from gtts import gTTS
import os
# Create a gTTS object
tts = gTTS(text)
# Save the generated speech to an audio file
tts.save("ramoutput.mp3")



In [ ]:
from pydub import AudioSegment

In [ ]:
import json

with open('/content/mocked_response-2.json', 'r') as f:
    data = json.load(f)

In [ ]:
strings = []
for cnt, i in enumerate(data['submodelElements'][0]['value'][3]['value']) :
    string = ""
    if cnt == 0:
      string = string + f'lets look at your first task.  '
    if cnt == 1:
      string = string + f'lets look at your second task.  '
    for j in i['value']:
        if j['idShort']=='TaskName':
            string = string + 'Task Name is ' + j['value'] + '. '
        if j['idShort']== 'TaskDetails':
            string = string + ' Task Details are as follows. '+j['value'] + '. '

    strings.append(string)


print(strings)


['lets look at your first task.  Task Name is Garment and Equipment Preparation.  Task Details are as follows. Prepare by wearing sterile garments (gown, gloves, mask) and setting up a laminar flow hood or isolator to maintain aseptic conditions. ',
 'lets look at your second task.  Task Name is Aseptic Filling.  Task Details are as follows. Perform the sterile filling of vials, ampoules, or containers with the product using aseptic techniques to maintain sterility throughout the process.. ']

In [ ]:
for cnt, string in enumerate(strings):
  tts = gTTS(string)
  tts.save(f"json_music{cnt+1}.mp3")

segment1 = AudioSegment.from_mp3("/content/json_music1.mp3")
segment2 = AudioSegment.from_mp3("/content/json_music2.mp3")

# Add 3 seconds of silence between the segments
three_seconds_silence = AudioSegment.silent(duration=5000)  # 3000 milliseconds = 3 seconds
combined_audio = segment1 + three_seconds_silence + segment2

# Export the combined audio to an output file
combined_audio.export("combined_audio.mp3", format="mp3")

<_io.BufferedRandom name='combined_audio.mp3'>